In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mesa_reader as mr

# Conversion factor from grams to solar masses
GRAMS_TO_SOLAR_MASSES = 1 / 1.989e33

# List of isotopes to analyse
ISOTOPES = ['c12', 'c13', 'n14', 'n15', 'o16', 'ne20', 'na23', 'mg24', 'al27', 'si28']

# Initial masses to plot
INITIAL_MASSES = list(range(10, 21)) + [25]

# File paths for each mass (update these as needed)
FILE_PATHS = ["/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_10M_3.4CBM_MLON/LOGS/profile911.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_11M_3.7CBM_MLON/LOGS/profile511.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_12M_4CBM_MLON/LOGS/profile386.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_13M_4.3CBM_MLON/LOGS/profile397.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_14M_4.6CBM_MLON/LOGS/profile364.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_15M_5CBM_MLON/LOGS/profile355.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_16M_5CBM_MLON/LOGS/profile374.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_17M_5CBM_MLON/LOGS/profile367.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_18M_5CBM_MLON/LOGS/profile452.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_19M_5CBM_MLON/LOGS/profile357.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_20M_5CBM_MLON/LOGS/profile385.data",
              "/lupus2/muthu/mesa_models/Z0/M128_shyne2/M128_25M_5CBM_MLON/LOGS/profile544.data",]

def load_data(file_path, isotope_attr, dm_attr):
    data = mr.MesaData(file_path)
    
    # Load the isotope abundance and dM using the provided attribute names
    isotope_abundance = getattr(data, isotope_attr, None)
    dM = getattr(data, dm_attr, None)
    
    if isotope_abundance is None or dM is None:
        raise ValueError(f"Attributes '{isotope_attr}' or '{dm_attr}' not found in the data.")
    
    # Convert dM from grams to solar masses
    dM_solar = dM * GRAMS_TO_SOLAR_MASSES
    
    return isotope_abundance, dM_solar

def calculate_mass_of_isotope(isotope_abundance, dM_solar):
    # Calculate the mass of the isotope by integrating isotope_abundance * dM_solar
    return np.trapz(isotope_abundance * dM_solar)

def plot_isotopes_vs_mass(file_paths, masses, isotope_attr, dm_attr):
    plt.figure(figsize=(12, 8), dpi=300)
    plt.title(f'Mass of {isotope_attr.upper()} vs Initial Stellar Mass', color='k')
    plt.xlabel('Initial Stellar Mass [$M_\odot$]', color='k')
    plt.ylabel(f'Mass of {isotope_attr.upper()} [$M_\odot$]', color='k')
    plt.grid(True, color='k', linestyle=':')
    plt.yscale('log')

    isotope_masses = []
    
    for file_path in file_paths:
        try:
            isotope_abundance, dM_solar = load_data(file_path, isotope_attr, dm_attr)
            isotope_mass = calculate_mass_of_isotope(isotope_abundance, dM_solar)
            isotope_masses.append(isotope_mass)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
            isotope_masses.append(np.nan)  # Handle missing data gracefully
    
    plt.plot(masses, isotope_masses, marker='o', label=f'{isotope_attr.upper()}', color='k')
    plt.legend(facecolor='w', edgecolor='k', labelcolor='k')
    plt.show()

# Plot for each isotope
dm_attr = 'dm'  # Replace with the actual attribute name for dM
for isotope in ISOTOPES:
    plot_isotopes_vs_mass(FILE_PATHS, INITIAL_MASSES, isotope, dm_attr)